<a href="https://colab.research.google.com/github/Aaditya18SP/MSC_Data_Science/blob/windows/prac9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import socket
import time
import random
import threading

# Function to generate and send log data to the socket
def generate_logs():
    host = "localhost"
    port = 9999
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(5)
    print(f"Server listening on {host}:{port}...")

    while True:
        client_socket, addr = server_socket.accept()
        print(f"Connection established with {addr}")
        try:
            while True:
                # Simulate a log line (you can modify this as needed)
                log_line = f"192.168.0.{random.randint(1, 100)} - - [25/Jan/2025:10:{random.randint(0,59):02d}:{random.randint(0,59):02d}:00 +0000] \"GET /page{random.randint(1, 10)} HTTP/1.1\" 200 {random.randint(200, 1000)}"
                client_socket.send((log_line + "\n").encode())
                time.sleep(1)  # Send one log every second
        except Exception as e:
            print(f"Error: {e}")
            client_socket.close()

# Run the log generation in a separate thread
def start_log_server():
    threading.Thread(target=generate_logs, daemon=True).start()

if __name__ == "__main__":
    start_log_server()

Server listening on localhost:9999...


In [2]:
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext

In [3]:
spark = SparkSession.builder.master("local").appName("xyz").getOrCreate()
ssc = StreamingContext(spark.sparkContext, 1)

/usr/local/lib/python3.10/dist-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


In [4]:
lines = ssc.socketTextStream("localhost", 9999)
words = lines.flatMap(lambda line: line.split(" "))
pairs = words.map(lambda word: (word, 1))
wordCounts = pairs.reduceByKey(lambda x, y: x + y)

wordCounts.pprint()
#ssc.stop()
ssc.start()
ssc.awaitTermination()

Connection established with ('127.0.0.1', 35690)


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 